In [1]:
import numpy as np
import pandas as pd
import os
import math as m
import matplotlib.pyplot as plt
import time

## Import Data Functions

In [2]:
df = pd.read_csv(r"..\data\dataset_new_score_v3.csv").drop(columns="Unnamed: 0")
df.columns

Index(['titre', 'budget', 'recette', 'duree', 'realisateur', 'casting',
       'compagnies_production', 'suite', 'Comedy', 'Drama', 'Horror',
       'Thriller', 'Action', 'Adventure', 'Science Fiction', 'Fantasy',
       'Family', 'Crime', 'Romance', 'Animation', 'Mystery', 'War', 'Music',
       'Western', 'History', 'Documentary', 'TV Movie', 'benefice global',
       'saison', 'casting_score_sum', 'casting_score_mean',
       'casting_score_med', 'casting_score_min', 'casting_score_max',
       'director_score_sum', 'director_score_mean', 'director_score_med',
       'director_score_min', 'director_score_max', 'compagnies_score_sum',
       'compagnies_score_mean', 'compagnies_score_med', 'compagnies_score_min',
       'compagnies_score_max'],
      dtype='object')

## Datasets and Scaling

In [3]:
from sklearn.model_selection import train_test_split

TEST_RATIO = 0.9
 
X_train, X_test, Y_train, Y_test = train_test_split(df.drop(columns=["titre", "recette", "realisateur", "casting", "compagnies_production", "benefice global",'director_score_mean', 'director_score_med',
       'director_score_min', 'director_score_max', 'compagnies_score_mean', 'compagnies_score_med', 'compagnies_score_min',
       'compagnies_score_max']), df["benefice global"], test_size = TEST_RATIO, shuffle=True, random_state = 1)

In [4]:
from sklearn.preprocessing import StandardScaler

X_train_f = StandardScaler().fit(np.array(X_train)).transform(np.array(X_train))


X_test_f = StandardScaler().fit(np.array(X_test)).transform(np.array(X_test))

print("Done")

Done


## Model

In [5]:
from sklearn.ensemble import RandomForestRegressor

start = time.time()
regr = RandomForestRegressor(random_state=0)
regr.fit(X_train_f, Y_train)
end = time.time()
elapsed1 = end - start

print("Temps exécution : ", elapsed1)


Temps exécution :  1.3306324481964111


In [6]:
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.metrics import mean_absolute_error

#Get the time of execution
start = time.time()


score_mape = mean_absolute_percentage_error(Y_test, regr.predict(X_test_f))
score_mae = mean_absolute_error(Y_test, regr.predict(X_test_f))


end = time.time()
elapsed2 = elapsed1 + end - start

print("Temps exécution : ", elapsed2)
print("MAPE : ", score_mape)
print("MAE : ", score_mae)

Temps exécution :  1.574275255203247
MAPE :  2.6718079065813444e+16
MAE :  302.3839189312093


### GridSearch

In [9]:
from sklearn.model_selection import GridSearchCV

n_estimators = [100, 150]
min_samples_leaf = [k for k in range(1,5)]
min_samples_split = [2,3,4,5]
criterion = ["friedman_mse", "poisson"]
max_depth = [3,5,10, None]

params_grid = {'n_estimators': n_estimators, 'min_samples_leaf': min_samples_leaf,
            'min_samples_split': min_samples_split, "criterion":criterion, "max_depth":max_depth }

rf_clf = RandomForestRegressor(random_state=0)

rf_cv = GridSearchCV(rf_clf, params_grid, scoring="neg_mean_absolute_error", cv=10)
rf_cv.fit(X_train, Y_train)


print('GridSearch CV best score : {:.4f}\n\n'.format(rf_cv.best_score_))

# print parameters that give the best results
print('Parameters that give the best results :','\n\n', (rf_cv.best_params_))

# print estimator that was chosen by the GridSearch
print('\n\nEstimator that was chosen by the search :','\n\n', (rf_cv.best_estimator_))

KeyboardInterrupt: 

In [12]:
import sklearn

sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

Cross validation

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

#Get the time of execution

scorer = make_scorer(mean_absolute_error)
score = cross_val_score(regr, X_train_f, Y_train, cv=10, scoring=scorer)


print("%0.2f MAE with a standard deviation of %0.2f" % (score.mean(), score.std()))




scorer = make_scorer(mean_absolute_percentage_error)
score = cross_val_score(regr, X_train_f, Y_train, cv=10, scoring=scorer)



print("%0.2f MAPE with a standard deviation of %0.2f" % (score.mean(), score.std()))


## Tuning some shit

In [ ]:
K = 10 #crossvalidation
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.metrics import make_scorer

# Paramètres à piper so good
scorer = make_scorer(mean_absolute_percentage_error)
parameters = [{'max_depth': [k for k in range(3,10)], 'criterion': ["squared_error","absolute_error","poisson"]}]
print("ça tune de la thune")
regr_t = GridSearchCV(RandomForestRegressor(), parameters, cv = K, scoring = scorer)
regr_t.fit(X_test_f, Y_test)

# On check les scores
print("Grid scores")
means = regr_t.cv_results_['mean_test_score']
stds = regr_t.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, regr_t.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))